In [1]:
import os
import math
import json
import re
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn import tree
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
import joblib
import numpy as np
import pandas as pd
import random
from keybert import KeyBERT
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from collections import Counter
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
import tensorflow as tf
import datetime
from tensorflow.keras.models import load_model
from tensorflow import keras
from keras import layers
import torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
import gc

In [2]:
X_df_16 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2016_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, header = None)
X_df_16.fillna(0,inplace=True)
X_df_17 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2017_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
X_df_17.fillna(0,inplace=True)
X_df_18 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2018_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
X_df_18.fillna(0,inplace=True)
X_df_19 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2019_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
X_df_19.fillna(0,inplace=True)
X_df_20 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2020_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
X_df_20.fillna(0,inplace=True)
X_df_21 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2021_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
X_df_21.fillna(0,inplace=True)
X_df_22 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2022_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
X_df_22.fillna(0,inplace=True)
X_df_23 = pd.read_csv('Datasets/dsc_fc_summed_spectra_2023_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
X_df_23.fillna(0,inplace=True)
X_df = [X_df_16, X_df_17, X_df_18, X_df_19, X_df_20, X_df_21]
# X_df = [X_df_16]

In [3]:
Y_df = []
for i in range(len(X_df)):
    with open('Datasets/DGD/' + str(2016 + i) + '_DGD.txt', 'r') as file:
        lines = file.readlines()[12:]
    days = X_df[i][0].apply(lambda x: " ".join([x.strftime('%Y'), x.strftime('%m'), x.strftime('%d')])).values
    days = set(days)
    df = pd.DataFrame(columns=["DateTime", "MiddleLatitudeA", "MiddleLatitudeK", "HighLatitudeA", "HighLatitudeK", "EstimatedPlanetaryA", "EstimatedPlanetaryK"])
    for line in lines:
        ln = line.replace("-", " -").split()
        dt = " ".join(ln[:3])
        if dt not in days:
            continue
        MiddleLatitudeK = list()
        HighLatitudeK = list()
        EstimatedPlanetaryK = list()
        for i in range(8):
            df.loc[len(df)] = {"DateTime" : (dt + " " +str(i*3)), "MiddleLatitudeA": int(ln[3]), "MiddleLatitudeK": int(ln[4 + i]), "HighLatitudeA": int(ln[12]), "HighLatitudeK": int(ln[13 + i]), "EstimatedPlanetaryA": float(ln[21]), "EstimatedPlanetaryK": float(ln[22 + i])}
            # MiddleLatitudeK.append(int(ln[4 + i]))
            # HighLatitudeK.append(int(ln[13 + i]))
            # EstimatedPlanetaryK.append(int(ln[22 + i]))
        # Y_df.loc[len(Y_df)] = {"DateTime" : (dt + " " +str(i*3)), "MiddleLatitudeA": ln[3], "MiddleLatitudeK": MiddleLatitudeK, "HighLatitudeA": ln[12], "HighLatitudeK": HighLatitudeK, "EstimatedPlanetaryA": ln[21], "EstimatedPlanetaryK": MiddleLatitudeK}
    Y_df.append(df)

In [4]:
X_full = pd.concat(X_df)
y_full = pd.concat(Y_df)

In [5]:
class KIndexDataset(Dataset):
    def __init__(self, X, y, input_seq_len, output_seq_len):
        """
        Инициализация датасета.

        Параметры:
            X (pd.DataFrame): Минутные данные признаков.
            y (pd.DataFrame): Данные K-index по дням.
            input_seq_len (int): Длина входной последовательности.
            output_seq_len (int): Длина выходной последовательности (прогноза).
        """
        x_temp = list()
        y_temp = list()
        # self.X = torch.tensor(X.values, dtype=torch.float32)
        # self.y = torch.tensor([y[0] for y in y.values])
        self.input_seq_len = input_seq_len
        self.output_seq_len = output_seq_len
        for i in range(input_seq_len, int(len(y) / 8) - output_seq_len - 1):
            x_temp.append(X[1440 * (i - input_seq_len):1440 * (i)].values)
            y_temp.append(y[i * 8 : 8 * (i + output_seq_len)].values)
        print('a')
        self.X = torch.tensor(np.array(x_temp), dtype=torch.float32)
        self.y = torch.tensor(np.array(y_temp), dtype=torch.float32)
        del x_temp
        del y_temp
        gc.collect()

    def __len__(self):
        """
        Возвращает размер датасета.
        """
        return len(self.X) - self.input_seq_len - self.output_seq_len + 1

    def __getitem__(self, idx):
        """
        Возвращает пару (входная последовательность, целевая последовательность) по индексу.

        Параметры:
            idx (int): Индекс элемента датасета.
        """
        X_seq = self.X[idx]
        y_seq = self.y[idx]
        return X_seq, y_seq

In [6]:
# Гиперпараметры
input_seq_len = 336
output_seq_len = 168

# Создание экземпляра датасета
train_data = KIndexDataset(X_full[[1, 2, 3]], y_full[['EstimatedPlanetaryK']], input_seq_len, output_seq_len)

# # # Создание DataLoader
# batch_size = 32  # размер батча
# train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
# tmp = [i for i in iter(train_loader)]

a


In [7]:
batch_size = 32  # размер батча
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
tmp = [i for i in iter(train_loader)]

ValueError: __len__() should return >= 0

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1):
        super(Encoder, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

    def forward(self, src):
        _, (hidden, cell) = self.lstm(src)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, num_layers=1):
        super(Decoder, self).__init__()
        self.output_dim = output_dim
        self.lstm = nn.LSTM(output_dim, hidden_dim, num_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(1)
        output, (hidden, cell) = self.lstm(input, (hidden, cell))
        prediction = self.fc_out(output.squeeze(1))
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        trg_len = trg.shape[1]
        batch_size = trg.shape[0]
        trg_dim = trg.shape[2]

        outputs = torch.zeros(batch_size, trg_len, trg_dim).to(self.device)

        hidden, cell = self.encoder(src)

        input = trg[:, 0, :]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:, t, :] = output

            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            input = trg[:, t, :] if teacher_force else output

        return outputs

# Пример использования
INPUT_DIM = 3  # размер входного вектора
OUTPUT_DIM = 2  # размер выходного вектора
ENC_HID_DIM = 64  # размер скрытого состояния в encoder
DEC_HID_DIM = 64  # размер скрытого состояния в decoder

enc = Encoder(INPUT_DIM, ENC_HID_DIM)
dec = Decoder(OUTPUT_DIM, DEC_HID_DIM)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = Seq2Seq(enc, dec, device).to(device)

# # Затем вы можете использовать оптимизатор и критерий потерь, подходящие для вашей задачи, чтобы обучить модель.
# optimizer = optim.Adam(model.parameters())
# criterion = nn.MSELoss()  # Возможно, вы хотите использовать другой критерий потерь, в зависимости от вашей задачи
model

In [ ]:
batch_size = 32
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
# torch.cpu.empty_cache()

# Параметры модели
input_dim = 3
output_dim = 2
enc_hid_dim = 64
dec_hid_dim = 64

# Инициализация модели, оптимизатора и функции потерь
encoder = Encoder(input_dim, enc_hid_dim)
decoder = Decoder(output_dim, dec_hid_dim)
model = Seq2Seq(encoder, decoder, device).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Обучение модели
num_epochs = 2

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for i, (x, y) in enumerate(tqdm(train_loader)):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()

        output = model(x, y)

        loss = criterion(output, y)
        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {epoch_loss / len(train_loader)}')

In [14]:

def createModel():
    # 1. Encoder setup
    encoder_inputs = layers.Input(shape=(None, 3))  # 'None' allows for variable sequence length
    encoder_lstm = layers.LSTM(16, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
    encoder_states = [state_h, state_c]

    # 2. Decoder setup
    decoder_inputs = layers.Input(shape=(None, 1))
    decoder_lstm = layers.LSTM(16, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = layers.Dense(1, activation='linear')  # Linear activation, but you can use others
    decoder_outputs = decoder_dense(decoder_outputs)

    decoder_state_input_h = layers.Input(shape=(16,))
    decoder_state_input_c = layers.Input(shape=(16,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_inf_outputs, state_h, state_c = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]

    decoder_inf_outputs = decoder_dense(decoder_inf_outputs)

    # Construct the model
    return keras.Model([encoder_inputs, decoder_inputs], decoder_outputs), keras.Model(encoder_inputs, encoder_states), keras.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_inf_outputs] + decoder_states)

model, encoder, decoder = createModel()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=keras.losses.mean_squared_error)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, 3)]    0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, None, 1)]    0           []                               
                                                                                                  
 lstm_2 (LSTM)                  [(None, 16),         1280        ['input_5[0][0]']                
                                 (None, 16),                                                      
                                 (None, 16)]                                                      
                                                                                            

In [6]:

def decode_sequence(input_seq, input_K):
    # Encode the input sequence to get initial states
    states_value = encoder.predict(input_seq)

    # Generate an empty target sequence with a size of 1.
    target_seq = np.zeros((1, 1, 1))  # Assuming 'm' is the dimension of the decoder's output.

    # Set the first token of target sequence with the start token.
    # Here, you'd typically use a tokenized representation.
    # For simplicity, we'll assume you have a function called `start_token_vector` that returns a vector representation of the <start> token.
    target_seq[0, 0] = input_K

    # Loop to decode the sequence
    stop_condition = False
    decoded = []
    while not stop_condition:
        output_tokens, h, c = decoder.predict([target_seq] + states_value)

        # Sample the next token (for simplicity, we're assuming a regression-like task here)
        sampled_token_vector = output_tokens[0, -1, :]

        decoded.append(sampled_token_vector)

        # Exit condition: either hit max length or find the <end> token.
        # Here, we'll just use a simple length-based condition for demonstration.
        if len(decoded) > 2920:
            stop_condition = True

        # Update the target sequence for the next iteration
        target_seq = np.zeros((1, 1, 1))
        target_seq[0, 0] = sampled_token_vector

        # Update states
        states_value = [h, c]

    return decoded

In [6]:
X, y = X_full[[1, 2, 3]], y_full[['EstimatedPlanetaryK']]
input_seq_len = 28*3
output_seq_len = 28*1
x_temp = list()
y_temp = list()
for i in range(input_seq_len, int(len(y) / 8) - output_seq_len - 1):
    x_temp.append(X[1440 * (i - input_seq_len):1440 * (i)].values)
    y_temp.append(y[i * 8: 8 * (i + output_seq_len)].values)
print('a')
X_train = tf.cast(np.array(x_temp), tf.float16)
y_train = tf.cast(np.array(y_temp), tf.float16)

a


In [7]:
input_seq_length = 120960
input_features = 3
output_seq_length = 224
latent_dim = 16

# Define the encoder
encoder_inputs = layers.Input(shape=(input_seq_length, input_features))
encoder_lstm = layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = layers.RepeatVector(output_seq_length)(encoder_outputs)
decoder_lstm = layers.LSTM(latent_dim, return_sequences=True)
decoder_outputs = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_outputs = layers.Dense(16, activation='relu')(decoder_outputs)
decoder_outputs = layers.Dense(8, activation='relu')(decoder_outputs)
decoder_outputs = layers.Dense(1, activation='relu')(decoder_outputs)

# Construct the model
model = tf.keras.Model(encoder_inputs, decoder_outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.1), loss='mse')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 120960, 3)]  0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 16),         1280        ['input_1[0][0]']                
                                 (None, 16),                                                      
                                 (None, 16)]                                                      
                                                                                                  
 repeat_vector (RepeatVector)   (None, 224, 16)      0           ['lstm[0][0]']                   
                                                                                              

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=64)


Epoch 1/10
27/27 [==============================] - 1868s 69s/step - loss: 1.4470
Epoch 2/10
27/27 [==============================] - 2610s 97s/step - loss: 1.4405
Epoch 3/10
27/27 [==============================] - 2735s 102s/step - loss: 1.4349
Epoch 4/10
18/27 [===================>..........] - ETA: 14:40 - loss: 1.4281

In [ ]:

model.save('models/model0')

In [9]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found